In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("meowmeowmeowmeowmeow/gtsrb-german-traffic-sign")

print("Path to dataset files:", path)

/Users/ihsanbolum/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


100%|██████████| 612M/612M [00:53<00:00, 12.0MB/s] 

Extracting files...


Path to dataset files: /Users/ihsanbolum/.cache/kagglehub/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/versions/1


In [8]:
path = "/Users/ihsanbolum/.cache/kagglehub/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/versions/1"
import os

In [13]:
print(f"files {os.listdir(path)}")

files ['Test', 'Test.csv', 'Meta', 'Train', 'Meta.csv', 'Train.csv']


In [ ]:
train_folder_path = os.path.join(path, "Train")
print(f"Train folder files: {os.listdir(train_folder_path)}") 

Train folder files: ['32', '35', '34', '33', '20', '18', '27', '9', '0', '11', '7', '29', '16', '42', '6', '28', '17', '1', '10', '19', '26', '8', '21', '38', '36', '31', '30', '37', '39', '41', '24', '23', '4', '15', '3', '12', '40', '2', '13', '5', '14', '22', '25']
/Users/ihsanbolum/.cache/kagglehub/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign/versions/1


In [18]:
class_0_path = os.path.join(train_folder_path, "0")
class_1_path = os.path.join(train_folder_path, "1")
       

In [ ]:
print(os.listdir(class_0_path[0]))

TypeError: unhashable type: 'list'